# Mega similarity check #

- Not scientific. Just merging random codes together

In [1]:
from pathlib import Path
import click
import torch
#import numpy as np
import json 
from safetensors.torch import load_file

In [2]:
g_device = "cpu"

In [3]:
def load_model(path: Path, device: str, print_ptl_info=False) -> dict[str, torch.Tensor]:
    if ".safetensors" in path.suffixes:
        return load_file(path, device=device)
    else:
        ckpt = torch.load(path, map_location=device)
        if print_ptl_info and "epoch" in ckpt and "global_step" in ckpt:
            print(f"[I] {path.name}: epoch {ckpt['epoch']}, step {ckpt['global_step']}")
        return ckpt["state_dict"] if "state_dict" in ckpt else ckpt


def check_equal_shape(a: torch.Tensor, b: torch.Tensor, fn):
    if a.shape != b.shape:
        return float('nan') #"DIFFERENT SHAPE"

    return fn(a.type(torch.float),b.type(torch.float))
    #return fn(a.reshape(-1), b.reshape(-1))


METRIC_MAP = {
    "equal": torch.equal,
    "l1": lambda a, b: check_equal_shape(a, b, lambda a, b: torch.dist(a, b, p=1)),
    "l2": lambda a, b: check_equal_shape(a, b, lambda a, b: torch.dist(a, b, p=2)),
    "cossim": lambda a, b: check_equal_shape(a, b, lambda a, b: torch.mean(torch.cosine_similarity(a, b, dim=0)))
}

In [4]:
def cmp_c(a_path, b_path, device, metric, no_ptl_info):
    metric_fn = METRIC_MAP[metric]
    
    try:
        a_path = a_path.decode('UTF-8')
        b_path = b_path.decode('UTF-8')
    except:
        #No need
        pass

    a = load_model(Path(a_path), device, not no_ptl_info)
    b = load_model(Path(b_path), device, not no_ptl_info)

    ak = set(a.keys())
    bk = set(b.keys())
    
    keys_inter = ak.intersection(bk)
    da = list(ak.difference(bk))
    db = list(bk.difference(ak))
    kv = {}
    for k in keys_inter:
        kv[k] = metric_fn(a[k], b[k]).numpy().tolist()
    return kv, da, db

In [5]:
def cmp_json(pa,pb,ofp,d="l2",npi=True):
    kv, da, db = cmp_c(Path(pa), Path(pb), g_device, d, npi)
    dj = {'kv':kv, 'da':da, 'db':db}
    with open(ofp, "w") as outfile:
        json.dump(dj, outfile, indent=4, sort_keys=True)

In [6]:
# Testing: Obvious result
cmp_json("./cosmoany.safetensors", "./cosmoany.safetensors", "ignore/test.json")

In [7]:
# VAE: WD only
pa = "../../stable-diffusion-webui/tmp/VAE/kl-f8-anime.ckpt"
pb = "../../stable-diffusion-webui/tmp/VAE/kl-f8-anime2.ckpt"
ofp = "ignore/vae_wd.json"
cmp_json(pa, pb, ofp)

In [8]:
# VAE: SD vs WD
pa = "../../stable-diffusion-webui/tmp/VAE/vae-ft-mse-840000-ema-pruned.ckpt"
pb = "../../stable-diffusion-webui/tmp/VAE/kl-f8-anime2.ckpt"
ofp = "ignore/vae_sdwd.json"
cmp_json(pa, pb, ofp)

In [9]:
# VAE: SD vs NAI
pa = "../../stable-diffusion-webui/tmp/VAE/vae-ft-mse-840000-ema-pruned.ckpt"
pb = "../../stable-diffusion-webui/tmp/VAE/animevae.pt"
ofp = "ignore/vae_sdnai.json"
cmp_json(pa, pb, ofp)

In [10]:
# VAE: WD vs NAI
pa = "../../stable-diffusion-webui/tmp/VAE/kl-f8-anime2.ckpt"
pb = "../../stable-diffusion-webui/tmp/VAE/animevae.pt"
ofp = "ignore/vae_wdnai.json"
cmp_json(pa, pb, ofp)

### Difference ###
- SD: None
- WD1: `encoder`, `decoder`
- WD2: `decoder`
- NAI: `decoder`

In [11]:
# SD: SD vs any3
pa = "../../stable-diffusion-webui/models/Stable-diffusion/sd-v1-4-full-ema.ckpt"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.ckpt"
ofp = "ignore/sd_sd_any3.json"
cmp_json(pa, pb, ofp)

In [12]:
# SD: SD v1.4 vs SD v1.5
pa = "../../stable-diffusion-webui/models/Stable-diffusion/sd-v1-2-full-ema.ckpt"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/sd-v1-4-full-ema.ckpt"
ofp = "ignore/sd_sd_sd.json"
cmp_json(pa, pb, ofp)

In [13]:
# SD: any3 vs basil
pa = "../../stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.ckpt"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/basil_mix.ckpt"
ofp = "ignore/sd_any3_basil.json"
cmp_json(pa, pb, ofp)

In [14]:
# SD: nai vs ac
pa = "../../stable-diffusion-webui/tmp/nodelaileak/lastmodel.ckpt"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/ACertainty.ckpt"
ofp = "ignore/sd_nai_ac.json"
cmp_json(pa, pb, ofp)

In [17]:
# SD: any3 vs any4
pa = "../../stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.ckpt"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/anything-v4.0-pruned.safetensors"
ofp = "ignore/sd_any3_any4.json"
cmp_json(pa, pb, ofp)

### Difference ###
- SD: None
- NAI: Some `cumprod` layers dropped
- ANY3: Same as NAI (merged)
- AC: Same as NAI (???)

In [15]:
# SD: nti vs bp
pa = "../../stable-diffusion-webui/models/Stable-diffusion/NMFSAN2-ep20.safetensors"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/bp_nman_e29.safetensors"
ofp = "ignore/sd_bp_nti.json"
cmp_json(pa, pb, ofp)

In [16]:
# SD: nti vs any4
pa = "../../stable-diffusion-webui/models/Stable-diffusion/NMFSAN2-ep20.safetensors"
pb = "../../stable-diffusion-webui/models/Stable-diffusion/anything-v4.0-pruned.safetensors"
ofp = "ignore/sd_nti_any4.json"
cmp_json(pa, pb, ofp)

### Special case ###
- NTI: No `text_model` = `clip-vit-large-patch14`